In [1]:
import sys

from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics

#path of train/test data folder
imdb_train_data_folder = "./aclImdb/train"
imdb_test_data_folder = "./aclImdb/test"

#loads the data from folder
imdb_train_data = load_files(imdb_train_data_folder,shuffle=False)
print("train_samples: %d" % len(imdb_train_data.data))

imdb_test_data = load_files(imdb_test_data_folder,shuffle=False)
print("test_samples: %d" % len(imdb_test_data.data))


train_samples: 25000
test_samples: 25000


In [2]:
print(imdb_train_data.data[0:40])
print(imdb_train_data.target)

[b"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.", b"Airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP's to his estate in preparation of it being opened to the public as a museum, also on board is Stevens daughter Julie (Kathleen Quinlan) & her son

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

#Builds a dictionary of features and transforms 
#documents to feature vectors: 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(imdb_train_data.data)
X_train_counts.shape

(25000, 74849)

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer

#transform our count-matrix to a tf-idf representation
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(25000, 74849)

In [ ]:
#Startbuiding your pipeline, ie: 
# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
